In [15]:
#Cargar

!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

from pyspark.sql import SparkSession
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
53 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

In [16]:
# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Movie_PIA") \
    .getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# cargar el archivo limpio que ya está en mi drive

file_path = '/content/drive/My Drive/movies_clean_final.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

In [ ]:
#Revisar que columnas tienen nulos
from pyspark.sql.functions import col, when, sum

null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) for c in df.columns])

null_counts.show()

+--------+-----+------------+----------+------+------------+-------+-----+------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+------------+
|movie_id|title|vote_average|vote_count|status|release_date|revenue|adult|budget|original_language|original_title|overview|popularity|genres|production_companies|production_countries|spoken_languages|year_release|
+--------+-----+------------+----------+------+------------+-------+-----+------+-----------------+--------------+--------+----------+------+--------------------+--------------------+----------------+------------+
|       0|    0|           0|         0|     0|           0|      0|    0|     0|                0|             0|     217|         5|     5|                   5|                   5|               5|           5|
+--------+-----+------------+----------+------+------------+-------+-----+------+-----------------+--------------+--------+----------+------+---

In [ ]:
#Limpieza
df = df.dropna()

In [ ]:
df.printSchema()

root
 |-- movie_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- vote_average: double (nullable = true)
 |-- vote_count: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- revenue: long (nullable = true)
 |-- adult: boolean (nullable = true)
 |-- budget: integer (nullable = true)
 |-- original_language: string (nullable = true)
 |-- original_title: string (nullable = true)
 |-- overview: string (nullable = true)
 |-- popularity: double (nullable = true)
 |-- genres: string (nullable = true)
 |-- production_companies: string (nullable = true)
 |-- production_countries: string (nullable = true)
 |-- spoken_languages: string (nullable = true)
 |-- year_release: integer (nullable = true)



In [ ]:
df.show(5)

+--------+---------------+------------+----------+--------+------------+----------+-----+---------+-----------------+---------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------+
|movie_id|          title|vote_average|vote_count|  status|release_date|   revenue|adult|   budget|original_language| original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages|year_release|
+--------+---------------+------------+----------+--------+------------+----------+-----+---------+-----------------+---------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------+
| 4520010|      Inception|       8.364|     34495|Released|   7/15/2010| 825532764|false|160000000|               en|      Inception|\Cobb a skilled t...|    83.952|Action Science Fi...|Legendary Picture...|Unit

In [ ]:
from pyspark.sql.functions import col, when

#Crear una columna de "mas votadas" con valor de 1 y cero el resto
df1 = df.withColumn("high_rating", when(col("vote_average") >= 7.0, 1).otherwise(0))

#seleccionar columnas para el modelo
features = ["budget", "popularity", "revenue", "vote_count", "high_rating"]

#filtrar el df
df1 = df1.select(features)

#Quitar valores nulos
df1 = df1.dropna()

In [ ]:
#dividir el dataset entre conjunto de entrenamiento y prueba

train_data1, test_data1 = df1.randomSplit([0.8, 0.2], seed=42)


Regresión logística para estimar si una pelicula será "altamente votada" o no

In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler

# Crear el vector de características
assembler = VectorAssembler(inputCols=['budget', 'popularity', 'revenue', 'vote_count'], outputCol='features')
train_data1 = assembler.transform(train_data1)

# Crear el modelo
lr = LogisticRegression(featuresCol='features', labelCol='high_rating')

# Entrenar el modelo
model1 = lr.fit(train_data1)

In [ ]:
# Preparar los datos de prueba
test_data1 = assembler.transform(test_data1)

# Hacer predicciones
predictions = model1.transform(test_data1)

# Mostrar resultados
predictions.select("high_rating", "prediction").show(10)

# Evaluación
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator(labelCol="high_rating", rawPredictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print(f"Precisión del modelo: {accuracy:.2f}")

+-----------+----------+
|high_rating|prediction|
+-----------+----------+
|          1|       0.0|
|          1|       0.0|
|          0|       0.0|
|          0|       0.0|
|          0|       0.0|
|          0|       0.0|
|          0|       0.0|
|          0|       0.0|
|          0|       0.0|
|          0|       0.0|
+-----------+----------+
only showing top 10 rows

Precisión del modelo: 0.51


In [ ]:
model1.save("logistic_regression_model")

In [ ]:
import joblib
import numpy as np

# Extraer los coeficientes y el intercepto
coef = np.array(model1.coefficients)
intercept = np.array(model1.intercept)

# Guardar coeficientes y el intercepto utilizando joblib
joblib.dump((coef, intercept), "logistic_regression_coefficients.pkl")

['logistic_regression_coefficients.pkl']

Segundo modelo



In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml import Pipeline

In [ ]:
df2 = df.select("movie_id", "title", "budget", "popularity", "revenue", "vote_average", "vote_count")
# Seleccionar y procesar columnas numéricas
numeric_columns = ["vote_average", "vote_count", "revenue", "budget", "popularity"]

# Crear un vector de características y escalarlo
assembler = VectorAssembler(inputCols=numeric_columns, outputCol="features")
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

# Pipeline para preprocesamiento
pipeline = Pipeline(stages=[assembler, scaler])
df_preprocessed = pipeline.fit(df2).transform(df2)

# Mostrar datos preprocesados
df_preprocessed.select("movie_id", "title", "scaled_features").show(truncate=False)

+--------+-------------------------------------------------+-----------------------------------------------------------------------------------------------+
|movie_id|title                                            |scaled_features                                                                                |
+--------+-------------------------------------------------+-----------------------------------------------------------------------------------------------+
|4520010 |Inception                                        |[2.6631819085474744,52.521391172026746,24.82587604873595,17.543447948063786,5.532013478671557] |
|4520011 |Interstellar                                     |[2.6800576427838463,49.59194758266656,21.10278725162009,18.09168069644078,9.241174745835453]   |
|4520012 |The Dark Knight                                  |[2.7103066003773435,46.619871758118194,30.20963491941436,20.284611689948754,8.608714942992284] |
|4520013 |Avatar                                          

In [ ]:
from pyspark.ml.linalg import DenseVector

def compare_movies(movie_id_1, movie_id_2):
    # Seleccionar las películas
    movie_1 = df_preprocessed.filter(F.col("movie_id") == movie_id_1).select("title", "scaled_features").first()
    movie_2 = df_preprocessed.filter(F.col("movie_id") == movie_id_2).select("title", "scaled_features").first()

    if not movie_1 or not movie_2:
        raise ValueError("Uno o ambos movie_id no existen en el conjunto de datos.")

    # Extraer atributos escalados
    title_1, features_1 = movie_1["title"], movie_1["scaled_features"]
    title_2, features_2 = movie_2["title"], movie_2["scaled_features"]

    # Calcular las diferencias entre los atributos
    differences = [abs(f1 - f2) for f1, f2 in zip(features_1, features_2)]
    comparison = {
        "movie_1": title_1,
        "movie_2": title_2,
        "differences": differences,
        "attributes": numeric_columns
    }
    return comparison

# Ejemplo de comparación
comparison_result = compare_movies(4520010, 4520011)
print(comparison_result)

{'movie_1': 'Inception', 'movie_2': 'Interstellar', 'differences': [0.016875734236371898, 2.929443589360183, 3.72308879711586, 0.5482327483769929, 3.7091612671638963], 'attributes': ['vote_average', 'vote_count', 'revenue', 'budget', 'popularity']}


In [ ]:
import pickle
from typing import Dict

class MovieComparisonModel:
    def __init__(self, df_preprocessed, numeric_columns):
        """
        Inicializa el modelo de comparación de películas.

        Args:
            df_preprocessed: DataFrame de PySpark preprocesado con columnas escaladas.
            numeric_columns: Lista de nombres de columnas numéricas para la comparación.
        """
        self.df_preprocessed = df_preprocessed
        self.numeric_columns = numeric_columns

    def find_movie_by_title(self, title: str) -> Dict:
        """
        Busca una película por su título.

        Args:
            title: Título de la película.

        Returns:
            Diccionario con el ID y los atributos de la película.
        """
        movie = self.df_preprocessed.filter(F.col("title") == title).first()
        if not movie:
            raise ValueError(f"Película con título '{title}' no encontrada.")
        return {"movie_id": movie["movie_id"], "title": movie["title"], "scaled_features": movie["scaled_features"]}

    def compare_movies_by_title(self, title_1: str, title_2: str) -> Dict:
        """
        Compara dos películas por su título.

        Args:
            title_1: Título de la primera película.
            title_2: Título de la segunda película.

        Returns:
            Diccionario con la comparación de atributos.
        """
        movie_1 = self.find_movie_by_title(title_1)
        movie_2 = self.find_movie_by_title(title_2)

        differences = [
            abs(f1 - f2) for f1, f2 in zip(movie_1["scaled_features"], movie_2["scaled_features"])
        ]

        return {
            "movie_1": movie_1["title"],
            "movie_2": movie_2["title"],
            "differences": differences,
            "attributes": self.numeric_columns,
        }

In [ ]:
import joblib
import numpy as np
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pickle

df2 = df.select("movie_id", "title", "budget", "popularity", "revenue", "vote_average", "vote_count")

# Selección de columnas para el modelo
numeric_columns = ["budget", "revenue", "vote_average", "vote_count", "popularity"]

# Usar VectorAssembler para combinar las columnas numéricas en un solo vector
assembler = VectorAssembler(inputCols=numeric_columns, outputCol="features")
df2 = assembler.transform(df2)

# Escalar los valores de las características usando MinMaxScaler (opcional pero útil para la comparación)
from pyspark.ml.feature import MinMaxScaler
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(df2)
df_scaled = scaler_model.transform(df2)

# Extraer los coeficientes (características escaladas) para compararlas
df_scaled = df_scaled.select("movie_id", "title", "scaled_features")
movies = df_scaled.collect()

# Crear un diccionario con los coeficientes
coef_dict = {}
for movie in movies:
    coef_dict[movie["title"]] = np.array(movie["scaled_features"])

# Guardar los coeficientes en un archivo .pkl
with open("movie_comparison_coefficients.pkl", "wb") as f:
    joblib.dump(coef_dict, f)

print("Modelo guardado como 'movie_comparison_coefficients.pkl'.")

Modelo guardado como 'movie_comparison_coefficients.pkl'.


In [ ]:
from pyspark.sql.functions import col, max


df3 = df

df3.show(5)
df3.printSchema()


+--------+---------------+------------+----------+--------+------------+----------+-----+---------+-----------------+---------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------+
|movie_id|          title|vote_average|vote_count|  status|release_date|   revenue|adult|   budget|original_language| original_title|            overview|popularity|              genres|production_companies|production_countries|    spoken_languages|year_release|
+--------+---------------+------------+----------+--------+------------+----------+-----+---------+-----------------+---------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+------------+
| 4520010|      Inception|       8.364|     34495|Released|   7/15/2010| 825532764|false|160000000|               en|      Inception|\Cobb a skilled t...|    83.952|Action Science Fi...|Legendary Picture...|Unit

In [ ]:
# Paso 1: Obtener la película con mayor budget por año
max_budget = df3.groupBy("year_release").agg(max("budget").alias("max_budget"))
merged_budget = df3.join(max_budget, (df3.year_release == max_budget.year_release) & (df3.budget == max_budget.max_budget), "inner") \
                   .select(df3["*"], max_budget["max_budget"])  # Selecciona columnas de df y max_budget

# Paso 2: Obtener la película con mayor revenue por año
max_revenue = df3.groupBy("year_release").agg(max("revenue").alias("max_revenue"))
merged_revenue = merged_budget.join(max_revenue, (merged_budget.year_release == max_revenue.year_release) & (merged_budget.revenue == max_revenue.max_revenue), "inner") \
                               .select(merged_budget["*"], max_revenue["max_revenue"])  # Selecciona columnas de merged_budget y max_revenue

# Paso 3: Obtener la película con mayor vote_average por año
max_vote_average = df3.groupBy("year_release").agg(max("vote_average").alias("max_vote_average"))
final_result = merged_revenue.join(max_vote_average, (merged_revenue.year_release == max_vote_average.year_release) & (merged_revenue.vote_average == max_vote_average.max_vote_average), "inner") \
                              .select(merged_revenue["*"], max_vote_average["max_vote_average"])  # Selecciona columnas de merged_revenue y max_vote_average

# Seleccionar solo las columnas necesarias para el resultado final
final_output = final_result.select(
    "year_release",
    "title",
    "budget",
    "revenue",
    "vote_average"
)

In [ ]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.ml.linalg import DenseVector
from pyspark.sql.functions import col, udf
from transformers import pipeline
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
from textblob import TextBlob

# Crear una sesión de Spark
spark = SparkSession.builder \
    .appName("Movie_PIA") \
    .getOrCreate()

from google.colab import drive
drive.mount('/content/drive')

# cargar el archivo limpio que ya está en mi drive

file_path = '/content/drive/My Drive/movies_clean_final.csv'
df = spark.read.csv(file_path, header=True, inferSchema=True)

# Definir una función para calcular el sentimiento
def analyze_sentiment(text):
    if text is None or text.strip() == "":
        return "Neutral"
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0:
        return "Positivo"
    elif polarity < 0:
        return "Negativo"
    else:
        return "Neutral"

# Registrar la función como UDF
sentiment_udf = udf(analyze_sentiment, StringType())

# Aplicar el análisis de sentimientos
df = df.withColumn("sentiment", sentiment_udf(col("overview")))

# Convertir a Pandas para guardar los resultados si es necesario
sentiment_results = df.select("title", "sentiment").toPandas()

# Guardar los resultados en un archivo .pkl
joblib.dump(sentiment_results["sentiment"].tolist(), "sentiment_analysis_results.pkl")

# Mostrar algunos ejemplos
df.select("title", "overview", "sentiment").show(10, truncate=False)


df = df.toPandas()

df.to_csv('movies_.csv', index=False)